In [32]:
import re 
input ='a(a|b)*bb+'
'''
This preporcessing is applied to the regex before applying shunting yard algorithm in order to : 
1) reduce all the regex to these operations only ( * , | , parenthes , concat )
2) add a concatination symbol between characters to be recognized by the algorithm as an operator 
3) replace every [match] with one char to be treated as other alphanumeric 
'''
def preprocessing(input : str) :
# input= 'a?a((cd)|(a|b))b+bb'
    removed_zero_or_one_symbol = re.sub(r'(\w)\?', r'(\1|)', input)
    removed_one_or_more_symbol = re.sub(r'(\w)\+', r'\1\1*', removed_zero_or_one_symbol)
    #TODO : store $  in the dict and replace with the value
    #TODO : Add $ to the alpha numeric in the forloop
    #TODO : Handle the dot operator 
    removed_match_brackets = re.sub(r'\[.*\]', r'$', removed_one_or_more_symbol) # for now one match 
    print(removed_match_brackets) 

    removed_match_brackets = removed_match_brackets + '^'
    i = 0 
    while (1):
        if ( removed_match_brackets [i] == '^' ) : break
        if  ( removed_match_brackets[i+1].isalnum() or removed_match_brackets[i+1] == '(')  and ( removed_match_brackets[i].isalnum() or removed_match_brackets[i] == ')' or removed_match_brackets[i] == '*' ) :
            removed_match_brackets = removed_match_brackets[:i+1] + '?' + removed_match_brackets[i+1:]
        i = i + 1
    print (removed_match_brackets)
    return removed_match_brackets

In [33]:
# Reeference : https://medium.com/@gregorycernera/converting-regular-expressions-to-postfix-notation-with-the-shunting-yard-algorithm-63d22ea1cf88
'''
Postfix notation removes the need for parentheses and allows computer programs to read in 
mathematical expressions one symbol after the other, instead of worrying about operator precedence 
and parentheses during computation. 

'''
def shuntingYard(input:str) :
    precedence_dict = {'*': 3, '?': 2, '|': 1}
    out =[]
    operator_stack = []
    for char in input :
        # If the input is alphanumeric then append to the output regex 
        if char.isalnum() :
            out.append(char)
        # If the input is an operator
        elif  char in precedence_dict.keys() :
            # The first operator in the stack 
            if len(operator_stack) == 0:
                operator_stack.append(char) 
            #  Any operator shouldn't be compared to an opening parenthes , if an opening parenthes is on the top of the stack Just add the char to the stack directly 
            elif operator_stack[-1] =='('or precedence_dict[ operator_stack[-1] ] < precedence_dict[char] :
                operator_stack.append(char)
            # If the operator on the top of the stack is the same as the current char then pop one to the output regex and leave the other in tha stack 
            # If two consecutive opening parenthes comes we need them both to be in the stack and not popped to the output because they will be deleted later 
            elif operator_stack[-1] != '(' and precedence_dict[ operator_stack[-1] ] == precedence_dict[char] : 
                out.append(char) 
            #If the operator at the top of the stack has higher precedence that the current operator then pop to the output until we can push the current operator to the stack 
            else : 
                while (len(operator_stack)>0 and operator_stack[-1] != '('and precedence_dict[ operator_stack[-1] ]  >= precedence_dict[char]) :
                    popped_operator = operator_stack.pop()
                    out.append(popped_operator)
                operator_stack.append(char) 
        elif char == '(' :
            operator_stack.append(char)
        # The current char is closing parenthes -> pop from the operator stack to the output until you reach an opening parenthes
        elif char == ')' :
            while operator_stack:
                operator = operator_stack.pop()
                if operator == '(':
                    break
                out.append(operator)
        # print ("parsing char :",char,"output:",out,"stack",operator_stack)
        # print("************************")
    # out.extend(operator_stack[::-1]) 
    # print ("Final out :",out)
    return out
    # print("Final stack :",operator_stack)


In [ ]:
import json
class Transition:
    def __init__(self, destination, input):
        self.destination = destination
        self.input = input


class State:
    state_counter = 0  

    def __init__(self):
        self.state_name = 'S'+ str(State.state_counter) 
        State.state_counter += 1
        self.transitions = [] 
        self.isTerminatingState = False 

    def add_transition(self, destination, input):
        self.transitions.append(Transition(destination, input))

    # def __repr__(self):
    #     return f"State(state_name={self.state_name}, transitions={self.transitions})"
    def to_dict(self):
        state_dict = {
            "isTerminatingState": self.isTerminatingState
        }
        for transition in self.transitions:
            state_dict[transition.input] = transition.destination
        return state_dict
class NFA:
    def __init__(self):
        self.states = []
        self.start_state = None
        self.accept_states = []

    def add_state(self, state , is_start = False , is_accept = False  ):
        self.states.append(state)
        if is_start:
            self.start_state = state
        if is_accept:
            self.accept_states.append(state)
        return state

    # def __repr__(self):
    #     states_str = "\n".join(str(state) for state in self.states)
    #     return (f"\nNFA(\n"
    #             f"  states=[\n{states_str}\n  ],\n"
    #             f"  start_state={self.start_state.state_name if self.start_state else None},\n"
    #             f"  accept_states={[state.state_name for state in self.accept_states]}\n)")
    # def to_dict(self):
    #     return {
    #         "startingState": self.start_state.state_name if self.start_state else None,
    #         '"'+()+'" :'
    #         "accept_states": [state.state_name for state in self.accept_states]
    #     }

    # def __repr__(self):
    #     return json.dumps(self.to_dict(), indent=2)
    def to_dict(self):
        nfa_dict = {
            "startingState": self.start_state.state_name if self.start_state else None
        }
        for state in self.states:
            nfa_dict[state.state_name] = state.to_dict()
        return nfa_dict

    def __repr__(self):
        return json.dumps(self.to_dict(), indent=2)

In [35]:
def alphanumeric_nfa(char) : 
    start_state = State() 
    end_state = State() 
    start_state.add_transition(end_state , char)
    nfa = NFA() 
    nfa.add_state(start_state , is_start = True )
    nfa.add_state(end_state , is_start = False , is_accept = True) 
    return nfa

def constructNFA(input : str) : 
    stack_NFA = []
    for char in input : 
        if char.isalnum() : 
            stack_NFA.append(alphanumeric_nfa(char)) 
    return stack_NFA 
def ThomsonsConstruction (input : str) : 
    return constructNFA(shuntingYard(preprocessing(input)))
nfa_result = ThomsonsConstruction('a')
for nfa in nfa_result:
    print(nfa) 

a
a^


AttributeError: 'Transition' object has no attribute 'transition_input'